In [21]:
import pandas as pd
import geopandas

In [22]:
import json

csv = pd.read_csv('csvallmetro.csv')
atlantageocodes = pd.DataFrame(csv['GEOID'].drop_duplicates())

# display(atlantageocodes)

In [23]:
# =SUM(SUMIFS(E:E,B:B,H2,D:D,J2,C:C,{"Less than $10,000","$10,000 to $19,999","$20,000 to $34,999"}))
# =SUM(SUMIFS(E:E,B:B,H9,D:D,J9,C:C,{"$35,000 to $49,999","$50,000 to $74,999"}))
# =SUM(SUMIFS(E:E,B:B,H16,D:D,J16,C:C,{"$75,000 to $99,999","$100,000 or more"}))

# =SUM(SUMIFS(D:D,A:A,G2,B:B,H2,C:C,{"Less than 20.0 percent","20.0 to 24.9 percent","25.0 to 29.9 percent"}))
# =SUM(SUMIFS(D:D,A:A,G3,B:B,H3,C:C,{"30.0 to 34.9 percent","35.0 to 39.9 percent","40.0 to 49.9 percent"}))
# =SUM(SUMIFS(D:D,A:A,G4,B:B,H4,C:C,{"50.0 percent or more"}))

income_mapping = {
        "Less than $10,000":'LowIncome',
        "$10,000 to $19,999":'LowIncome',
        "$20,000 to $34,999":'LowIncome',
            "$35,000 to $49,999":'MedIncome',
            "$50,000 to $74,999":'MedIncome',
                "$75,000 to $99,999":'HighIncome',
                "$100,000 or more":'HighIncome',
    }

rent_mapping = {
        "Less than 20.0 percent":'LowBurden',
        "20.0 to 24.9 percent":'LowBurden',
        "25.0 to 29.9 percent":'LowBurden',
            "30.0 to 34.9 percent":'MedBurden',
            "35.0 to 39.9 percent":'MedBurden',
            "40.0 to 49.9 percent":'MedBurden',
                "50.0 percent or more":'HighBurden',
    }

csv['IncomeG'] = csv['Income'].replace(income_mapping)
csv['RentG'] = csv['Rent as % of Income'].replace(rent_mapping)

In [24]:
# display(csv)

df = csv.groupby(['GEOID','YEAR','IncomeG','RentG']).sum()
df.to_csv('temp.csv')
df = pd.read_csv('temp.csv')

In [74]:
display(df.head())
type(df)




,GEOID,YEAR,IncomeG,RentG,HHs
0,13013180103,2014,High Income,High Burden,0
1,13013180103,2014,High Income,Low Burden,38
2,13013180103,2014,High Income,Med Burden,0
3,13013180103,2014,Low Income,High Burden,0
4,13013180103,2014,Low Income,Low Burden,0


pandas.core.frame.DataFrame

In [115]:
totalHHs = csv.groupby(['GEOID','YEAR']).sum()

display(totalHHs.head())

totalHHs.to_csv('temp.csv')
totalHHs = pd.read_csv('temp.csv')

display(totalHHs.head())

totalHHs = totalHHs[totalHHs['YEAR']==2017]



HHs
GEOID       YEAR     
13013180103 2014  109
            2015  157
            2016  162
            2017  219
13013180104 2014  116

,GEOID,YEAR,HHs
0,13013180103,2014,109
1,13013180103,2015,157
2,13013180103,2016,162
3,13013180103,2017,219
4,13013180104,2014,116


In [30]:
#Calculate % rent burdened all incomes
totalHHspercent = csv.groupby(['GEOID','YEAR','RentG','IncomeG']).sum()



totalHHspercent.to_csv('temp.csv')
totalHHspercent = pd.read_csv('temp.csv')



totalHHspercent.index = totalHHspercent['GEOID']
totalHHspercent.rename(columns={"GEOID": "GEOID2"}, inplace=True)

#display(totalHHspercent.head())

burdenlist = ['LowBurden', 'MedBurden', 'HighBurden']
incomelist = ['LowIncome', 'MedIncome', 'HighIncome']

for YEAR in [2014,2015,2016,2017]:
    for burden in burdenlist:
            for income in incomelist:
                totalHHscalc = totalHHspercent[(totalHHspercent['IncomeG']==income) & (totalHHspercent['YEAR']==YEAR) & (totalHHspercent['RentG']==burden)]['HHs'] / totalHHspercent[(totalHHspercent['YEAR']==YEAR)].groupby(['GEOID']).sum()['HHs']
                #totalHHscalc = totalHHspercent[(totalHHspercent['IncomeG']==income) & (totalHHspercent['YEAR']==YEAR) & (totalHHspercent['RentG']==burden)]
                
                
                
                #display(totalHHscalc)
                totalHHscalc = pd.DataFrame(totalHHscalc)
                totalHHscalc['GEOID'] = totalHHscalc.index
                totalHHscalc.rename(columns={"HHs": str(YEAR)+income+burden}, inplace=True)

                #display(totalHHscalc.head())
                with open('allga.geojson', 'r') as f:
                    data = json.load(f)


                for idx, row in totalHHscalc.iterrows():
                #     display(row)
                    GEOID = str(int(row['GEOID']))
                    #print(GEOID)
                    #YEAR = row['YEAR']
                    for feat in data['features']:
                        feat1 = feat['properties']['GEOID']
                #         print(GEOID)
                        #print(row.to_dict())
                        if feat1 == GEOID:
                            feat['properties'][str(YEAR)+income+burden] = str(row[str(YEAR)+income+burden])
                            #print('added' + GEOID)

                #Write result to a new file
                with open('allga.geojson', 'w') as f:
                    json.dump(data, f)

In [128]:
with open('allga.geojson', 'r') as f:
    data = json.load(f)


for idx, row in totalHHspercent.iterrows():
#     display(row)
    GEOID = str(int(row['GEOID']))
    print(GEOID)
    #YEAR = row['YEAR']
    for feat in data['features']:
        feat1 = feat['properties']['GEOID']
#         print(GEOID)
        #print(row.to_dict())
        if feat1 == GEOID:
            feat['properties']['HHPercentExtreme'] = str(row['HHPercentExtreme'])
            print('added' + GEOID)

#Write result to a new file
with open('allga.geojson', 'w') as f:
    json.dump(data, f)


13013180103
added13013180103
13013180104
added13013180104
13013180105
added13013180105
13013180106
added13013180106
13013180107
added13013180107
13013180108
added13013180108
13013180203
added13013180203
13013180204
added13013180204
13013180205
added13013180205
13013180206
added13013180206
13013180301
added13013180301
13013180302
added13013180302
13013180303
added13013180303
13013180401
added13013180401
13013180402
added13013180402
13013180501
added13013180501
13013180502
added13013180502
13013180503
added13013180503
13015960101
added13015960101
13015960102
added13015960102
13015960200
added13015960200
13015960300
added13015960300
13015960401
added13015960401
13015960402
added13015960402
13015960500
added13015960500
13015960600
added13015960600
13015960700
added13015960700
13015960801
added13015960801
13015960802
added13015960802
13015960803
added13015960803
13015960901
added13015960901
13015960902
added13015960902
13015961000
added13015961000
13045910101
added13045910101
13045910103
ad

13113140405
added13113140405
13113140406
added13113140406
13113140407
added13113140407
13113140408
added13113140408
13113140501
added13113140501
13113140502
added13113140502
13117130101
added13117130101
13117130102
added13117130102
13117130103
added13117130103
13117130104
added13117130104
13117130105
added13117130105
13117130201
added13117130201
13117130202
added13117130202
13117130203
added13117130203
13117130204
added13117130204
13117130205
added13117130205
13117130301
added13117130301
13117130302
added13117130302
13117130303
added13117130303
13117130304
added13117130304
13117130305
added13117130305
13117130306
added13117130306
13117130307
added13117130307
13117130403
added13117130403
13117130404
added13117130404
13117130405
added13117130405
13117130406
added13117130406
13117130408
added13117130408
13117130409
added13117130409
13117130410
added13117130410
13117130503
added13117130503
13117130504
added13117130504
13117130505
added13117130505
13117130506
added13117130506
13117130507
ad

added13135050520
13135050521
added13135050521
13135050522
added13135050522
13135050523
added13135050523
13135050524
added13135050524
13135050525
added13135050525
13135050526
added13135050526
13135050527
added13135050527
13135050528
added13135050528
13135050529
added13135050529
13135050530
added13135050530
13135050531
added13135050531
13135050532
added13135050532
13135050533
added13135050533
13135050534
added13135050534
13135050535
added13135050535
13135050536
added13135050536
13135050537
added13135050537
13135050538
added13135050538
13135050539
added13135050539
13135050540
added13135050540
13135050541
added13135050541
13135050542
added13135050542
13135050543
added13135050543
13135050544
added13135050544
13135050545
added13135050545
13135050546
added13135050546
13135050547
added13135050547
13135050548
added13135050548
13135050549
added13135050549
13135050605
added13135050605
13135050606
added13135050606
13135050607
added13135050607
13135050608
added13135050608
13135050609
added131350506

In [11]:
df.to_csv('final_summary.csv')

In [31]:
geotable = geopandas.read_file('allga.geojson')

display(geotable.head())
#geotable = geotable.astype('int64').dtypes
geotable[['GEOID']] = geotable[['GEOID']].astype('int64')

geotable2 = geotable.merge(atlantageocodes, on='GEOID')

# gdf = geopandas.GeoDataFrame(geotable2, geometry=geopandas.points_from_xy(df.Longitude, df.Latitude))

display(geotable2.head())

# geotable2 = geotable2[~geotable2.geometry.str.contains("MULTIPOLYGON")]


,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,...,2017LowIncomeLowBurden,2017MedIncomeLowBurden,2017HighIncomeLowBurden,2017LowIncomeMedBurden,2017MedIncomeMedBurden,2017HighIncomeMedBurden,2017LowIncomeHighBurden,2017MedIncomeHighBurden,2017HighIncomeHighBurden,geometry
0,13,153,020105,13153020105,201.05,Census Tract 201.05,G5020,S,5620037.0,89197.0,...,None,None,None,None,None,None,None,None,None,"POLYGON ((-83.660753 32.635764, -83.660735 32...."
1,13,153,020108,13153020108,201.08,Census Tract 201.08,G5020,S,10732046.0,65358.0,...,None,None,None,None,None,None,None,None,None,"POLYGON ((-83.69864699999999 32.659458, -83.69..."
2,13,089,023315,13089023315,233.15,Census Tract 233.15,G5020,S,23296834.0,363011.0,...,0.0,0.2170212765957447,0.15319148936170213,0.0,0.251063829787234,0.0,0.37872340425531914,0.0,0.0,"POLYGON ((-84.10821900000001 33.758834, -84.10..."
3,13,089,023421,13089023421,234.21,Census Tract 234.21,G5020,S,6124825.0,136878.0,...,0.0261172373766686,0.2727800348229832,0.15902495647127105,0.23331398723157284,0.08879860708067325,0.0,0.2199651770168311,0.0,0.0,"POLYGON ((-84.277354 33.674202, -84.277171 33...."
4,13,089,021504,13089021504,215.04,Census Tract 215.04,G5020,S,3732018.0,39771.0,...,0.0,0.14929056138186306,0.3078346699568168,0.06662553979025293,0.1727328809376928,0.0,0.30351634793337445,0.0,0.0,"POLYGON ((-84.338874 33.799947, -84.338553 33...."


,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,...,2017LowIncomeLowBurden,2017MedIncomeLowBurden,2017HighIncomeLowBurden,2017LowIncomeMedBurden,2017MedIncomeMedBurden,2017HighIncomeMedBurden,2017LowIncomeHighBurden,2017MedIncomeHighBurden,2017HighIncomeHighBurden,geometry
0,13,089,023315,13089023315,233.15,Census Tract 233.15,G5020,S,23296834.0,363011.0,...,0.0,0.2170212765957447,0.15319148936170213,0.0,0.251063829787234,0.0,0.37872340425531914,0.0,0.0,"POLYGON ((-84.10821900000001 33.758834, -84.10..."
1,13,089,023421,13089023421,234.21,Census Tract 234.21,G5020,S,6124825.0,136878.0,...,0.0261172373766686,0.2727800348229832,0.15902495647127105,0.23331398723157284,0.08879860708067325,0.0,0.2199651770168311,0.0,0.0,"POLYGON ((-84.277354 33.674202, -84.277171 33...."
2,13,089,021504,13089021504,215.04,Census Tract 215.04,G5020,S,3732018.0,39771.0,...,0.0,0.14929056138186306,0.3078346699568168,0.06662553979025293,0.1727328809376928,0.0,0.30351634793337445,0.0,0.0,"POLYGON ((-84.338874 33.799947, -84.338553 33...."
3,13,089,023419,13089023419,234.19,Census Tract 234.19,G5020,S,13219799.0,241092.0,...,0.061898211829436035,0.3658872077028886,0.218707015130674,0.2530949105914718,0.0,0.0,0.10041265474552957,0.0,0.0,"POLYGON ((-84.25414499999999 33.64708, -84.254..."
4,13,089,021411,13089021411,214.11,Census Tract 214.11,G5020,S,1667281.0,0.0,...,0.0,0.14579439252336449,0.5037383177570094,0.058878504672897194,0.13457943925233645,0.003738317757009346,0.11588785046728972,0.037383177570093455,0.0,"POLYGON ((-84.338921 33.860744, -84.3379610000..."


In [32]:
type(geotable2)
# geotable2.to_csv('testgeo.csv')
# geotable2 = geopandas.read_file('testgeo.csv')

geotable2.to_file("metroatlantaall.geojson", driver='GeoJSON')